In [ ]:
!pip install -U openai "pandas<2.2.3" gspread gspread-dataframe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.6/605.6 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.66.3
    Uninstalling openai-1.66.3:
      Successfully uninstalled openai-1.66.3


In [ ]:
#Import Modules
from openai import OpenAI
import pandas as pd
import os
import requests
from typing import Union, List, Dict
from enum import Enum
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    api_key='a0BIj000001iX7PMAU'
)

In [ ]:
DOC_ID = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
LINK = f"https://docs.google.com/spreadsheets/d/{DOC_ID}/export?format=xlsx"
resp = requests.get(LINK)
FILEPATH = "spreadsheet.xlsx"
with open(FILEPATH, 'wb') as file:
    file.write(resp.content)

In [ ]:
input_spreadsheet = pd.ExcelFile(FILEPATH)
products_df = pd.read_excel(input_spreadsheet, sheet_name='products')
emails_df = pd.read_excel(input_spreadsheet, sheet_name='emails')

In [ ]:
class RequestType(Enum):
    INQUIRY = "product inquiry"
    ORDER = "order request"

class Product:
    def __init__(self, row):
        self.id = row['product_id']
        self.name = row['name']
        self.category = row['category']
        self.description = row['description']
        self.stock = int(row['stock'])
        self.seasons = row['seasons']
        self.price = float(row['price'])

    def update_stock(self, quantity: int) -> bool:
        if self.stock >= quantity:
            self.stock -= quantity
            return True
        return False

class Inventory:
    def __init__(self, products_df):
        self.products = {row['product_id']: Product(row) for _, row in products_df.iterrows()}

    def get_product(self, product_id: str) -> Union[Product, None]:
        return self.products.get(product_id)

    def get_product_info(self, product_id: str) -> str:
        product = self.get_product(product_id)
        if product:
            return f"{product.name} ({product.category}): {product.description}, Stock: {product.stock}, Price: ${product.price}"
        return "Product not found."

inventory = Inventory(products_df)

In [ ]:
# Task 1: Classify Emails
def classify_emails(emails_df: pd.DataFrame) -> pd.DataFrame:
    classifications = []
    for _, row in emails_df.iterrows():
        email_id = row['email_id']
        prompt = (
            f"Classify the following email as either 'product inquiry' or 'order request' based on its intent:\n"
            f"Subject: {row['subject']}\nBody: {row['message']}\n"
            f"Respond with only the classification in quotes."
        )
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )
        category = response.choices[0].message.content.strip('"')
        classifications.append({'email ID': email_id, 'category': category})
    return pd.DataFrame(classifications)

email_classification_df = classify_emails(emails_df)

In [ ]:
# Task 2 : Process Order Requests
def process_orders(emails_df: pd.DataFrame, classifications_df: pd.DataFrame) -> tuple:
    order_status = []
    order_responses = []
    classifications_dict = dict(zip(classifications_df['email ID'], classifications_df['category']))

    for _, row in emails_df.iterrows():
        email_id = row['email_id']
        if classifications_dict[email_id] != RequestType.ORDER.value:
            continue

        # Improved prompt to enforce strict tuple output
        prompt = (
            f"Extract the product ID and quantity from this order request:\n"
            f"Subject: {row['subject']}\nBody: {row['message']}\n"
            f"Available products: {', '.join([f'{p.id}: {p.name}' for p in inventory.products.values()])}\n"
            f"Respond ONLY with a Python tuple in the format (product_id, quantity), where:\n"
            f"- product_id is a string or None if not found\n"
            f"- quantity is an integer, 'all' (for full stock), or None if not specified\n"
            f"Do NOT include any additional text or explanation."
        )
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )

        try:
            product_id, quantity_raw = eval(response.choices[0].message.content)
        except (SyntaxError, ValueError):
            product_id, quantity_raw = (None, None)  # Fallback if parsing fails

        if product_id:
            product = inventory.get_product(product_id)
            if product:
                # Handle quantity: convert 'all' to full stock, or use specified integer
                if isinstance(quantity_raw, str) and quantity_raw.lower() == 'all':
                    quantity = product.stock
                elif isinstance(quantity_raw, int) and quantity_raw > 0:
                    quantity = quantity_raw
                else:
                    quantity = None

                if quantity and product.update_stock(quantity):
                    status = "created"
                    response_text = (
                        f"Dear Customer,\n\nYour order for {quantity} {product.name}(s) has been successfully processed.\n"
                        f"Details: {product.description}, Total: ${product.price * quantity:.2f}\n"
                        f"Thank you for shopping with us!\n\nBest regards,\nFashion Store Team"
                    )
                else:
                    status = "out of stock"
                    response_text = (
                        f"Dear Customer,\n\nWe regret to inform you that your order for {quantity if quantity else 'unspecified quantity'} "
                        f"{product.name}(s) cannot be fulfilled due to insufficient stock. Current stock: {product.stock}.\n"
                        f"Please consider alternative products or contact us for restock updates.\n\nBest regards,\nFashion Store Team"
                    )
                order_status.append({'email ID': email_id, 'product ID': product_id, 'quantity': quantity, 'status': status})
                order_responses.append({'email ID': email_id, 'response': response_text})

    return pd.DataFrame(order_status), pd.DataFrame(order_responses)

# Re-run the function
order_status_df, order_response_df = process_orders(emails_df, email_classification_df)

# Display results for verification
print("Order Status:")
display(order_status_df.head())
print("Order Responses:")
display(order_response_df.head())

Order Status:


,email ID,product ID,quantity,status
0,E013,SLD7654,1.0,created
1,E019,CBT8901,NaN,out of stock
2,E023,CGN2345,5.0,out of stock


Order Responses:


,email ID,response
0,E013,"Dear Customer,\n\nYour order for 1 Slide Sanda..."
1,E019,"Dear Customer,\n\nWe regret to inform you that..."
2,E023,"Dear Customer,\n\nWe regret to inform you that..."


In [ ]:
# Task 3: Handle Product Inquiries
def handle_inquiries(emails_df: pd.DataFrame, classifications_df: pd.DataFrame) -> pd.DataFrame:
    inquiry_responses = []
    classifications_dict = dict(zip(classifications_df['email ID'], classifications_df['category']))

    for _, row in emails_df.iterrows():
        email_id = row['email_id']
        if classifications_dict[email_id] != RequestType.INQUIRY.value:
            continue

        prompt = (
            f"Extract the product ID or name from this inquiry:\n"
            f"Subject: {row['subject']}\nBody: {row['message']}\n"
            f"Available products: {', '.join([f'{p.id}: {p.name}' for p in inventory.products.values()])}\n"
            f"Respond with the product_id or name as a string, or 'None' if not found."
        )
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )
        identifier = response.choices[0].message.content.strip('"')

        product = inventory.get_product(identifier) if identifier in inventory.products else None
        if not product:
            for p in inventory.products.values():
                if identifier.lower() in p.name.lower():
                    product = p
                    break

        if product:
            response_text = (
                f"Dear Customer,\n\nThank you for your inquiry about {product.name}.\n"
                f"Details: {product.description}\nCategory: {product.category}\nStock: {product.stock}\nPrice: ${product.price}\n"
                f"Let us know if you need further assistance!\n\nBest regards,\nFashion Store Team"
            )
        else:
            response_text = (
                f"Dear Customer,\n\nThank you for your inquiry. We couldn’t find a product matching '{identifier}'.\n"
                f"Please check the product name or ID and feel free to reach out with more details!\n\nBest regards,\nFashion Store Team"
            )
        inquiry_responses.append({'email ID': email_id, 'response': response_text})

    return pd.DataFrame(inquiry_responses)

inquiry_response_df = handle_inquiries(emails_df, email_classification_df)

# Create and populate output Google Spreadsheet
output_document = gc.create('Solving Business Problems with AI - Output')

# Create 'email-classification' sheet
email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')
set_with_dataframe(email_classification_sheet, email_classification_df)

# Create 'order-status' sheet
order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')
set_with_dataframe(order_status_sheet, order_status_df)

# Create 'order-response' sheet
order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')
set_with_dataframe(order_response_sheet, order_response_df)

# Create 'inquiry-response' sheet
inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')
set_with_dataframe(inquiry_response_sheet, inquiry_response_df)

# Remove default sheet created by gspread
output_document.del_worksheet(output_document.sheet1)

# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')

# Output the shareable link
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")



Shareable link: https://docs.google.com/spreadsheets/d/17kk1CAp313M0hMjI6LnKjebw6xOXfm-rEs0m8et9czA


In [ ]:
# Display results for verification
print("Email Classifications:")
display(email_classification_df.head())
print("Order Status:")
display(order_status_df.head())
print("Order Responses:")
display(order_response_df.head())
print("Inquiry Responses:")
display(inquiry_response_df.head())

Email Classifications:


,email ID,category
0,E001,order request
1,E002,order request
2,E003,product inquiry
3,E004,order request
4,E005,product inquiry


Order Status:


,email ID,product ID,quantity,status
0,E013,SLD7654,1.0,created
1,E019,CBT8901,NaN,out of stock
2,E023,CGN2345,5.0,out of stock


Order Responses:


,email ID,response
0,E013,"Dear Customer,\n\nYour order for 1 Slide Sanda..."
1,E019,"Dear Customer,\n\nWe regret to inform you that..."
2,E023,"Dear Customer,\n\nWe regret to inform you that..."


Inquiry Responses:


,email ID,response
0,E003,"Dear Customer,\n\nThank you for your inquiry. ..."
1,E005,"Dear Customer,\n\nThank you for your inquiry a..."
2,E006,"Dear Customer,\n\nThank you for your inquiry a..."
3,E009,"Dear Customer,\n\nThank you for your inquiry a..."
4,E011,"Dear Customer,\n\nThank you for your inquiry a..."
